In [1]:
using ParallelTemperingMonteCarlo, StaticArrays, DelimitedFiles

Broadly we need to save:

-Params which do not change

-Configs (the BC may change?)

-Results

In [2]:
function writeconfig(savefile , config{N,BC,T}, ) where {N,BC,T}
    for row in config.pos
        write(savefile, "$(row[1]) $(row[2]) $(ro2[3]) \n")
    end
end


Beginning with the static parameters which do not change:

In [3]:
# function writeparams(savefile, params, temp)# , ensemble, pot)
#     write(savefile,"cycles: sample_rate: n_traj: n_atoms: t_i: t_f \n")
#     write(savefile, "$(params.mc_cycles) $(params.mc_sample) $(params.n_traj) $(params.n_atoms) $(first(temp.t_grid)) $(last(temp.t_grid)) \n")
# end

function writeparams(savefile,params,temp)
    headersvec = ["cycles:" "sample_rate:" "n_traj:" "n_atoms:" "t_i:" "t_f"]
    paramsvec = [params.mc_cycles params.mc_sample params.n_traj params.n_atoms first(temp.t_grid) last(temp.t_grid)]
    writedlm(savefile, [headersvec, paramsvec], ' ' )
end

writeparams (generic function with 1 method)

In [16]:
function writeensemble(savefile,ensemble::NVT)

    headersvec = ["ensemble" "n_atom_moves" "n_atom_swaps" ]
    valuesvec = ["NVT{$(ensemble.n_atoms)}" ensemble.n_atom_moves ensemble.n_atom_swaps]
    writedlm(savefile, [headersvec, valuesvec], ' ' )
end
function writeensemble(savefile,ensemble::NPT)

    headersvec = ["ensemble" "n_atom_moves" "n_volume_moves" "n_atom_swaps" ]
    valuesvec = ["NPT{$(ensemble.n_atoms)}" ensemble.n_atom_moves ensembles.n_volume_moves emsemble.n_atom_swaps]
    writedlm(savefile, [headersvec, valuesvec], ' ' )
end

writeensemble (generic function with 2 methods)

In [27]:
function writepotential(savefile,potential::Ptype) where Ptype <: AbstractDimerPotential
    coeff_vec = transpose([potential.coeff[i] for i in eachindex(potential.coeff)])

    write(savefile,"ELJ_coeffs: " )
    writedlm(savefile, coeff_vec, ' ')
end
function writepotential(savefile,potential::Ptype) where Ptype <: AbstractDimerPotentialB

    coeff_vec_a = transpose([potential.coeff_a[i] for i in eachindex(potential.coeff_a)])
    coeff_vec_b = transpose([potential.coeff_b[i] for i in eachindex(potential.coeff_b)])
    coeff_vec_c = transpose([potential.coeff_c[i] for i in eachindex(potential.coeff_c)])
    write(savefile,"ELJB \n")
    writedlm(savefile, [coeff_vec_a, coeff_vec_b, coeff_vec_c])
end
function writepotential(savefile,potential::Ptype) where Ptype <: EmbeddedAtomPotential
    write(savefile,"EAM: $(potential.n) $(potential.m) $(potential.ean) $(potential.eCam) \n")
end 
function writepotential(savefile,potential::Ptype) where Ptype <: AbstractMachineLearningPotential
    write("Define the potential elsewhere, it's too complicated for simple I/O \n")
end

writepotential (generic function with 4 methods)

In [28]:
function save_init(potential,ensemble,params,temp)
    if ispath("./checkpoint/params.data") == true
    else
        mkpath("./checkpoint/")
        paramsfile = open("./checkpoint/params.data","w+")
        
        writeparams(paramsfile,params,temp)
        writeensemble(paramsfile,ensemble)
        writepotential(paramsfile,potential)
        
        close(paramsfile)
    end
end


save_init (generic function with 1 method)

In [29]:
n_atoms = 13
ti = 4.
tf = 16.
n_traj = 25
temp = TempGrid{n_traj}(ti,tf) 
mc_cycles = 1000 #default 20% equilibration cycles on top
mc_sample = 1  #sample every mc_sample MC cycles
displ_atom = 0.1 # Angstrom
n_adjust = 100
max_displ_atom = [0.1*sqrt(displ_atom*temp.t_grid[i]) for i in 1:n_traj]
mc_params = MCParams(mc_cycles, n_traj, n_atoms, mc_sample = mc_sample, n_adjust = n_adjust)
c=[-10.5097942564988, 989.725135614556, -101383.865938807, 3918846.12841668, -56234083.4334278, 288738837.441765]
pot = ELJPotentialEven{6}(c)

a=[0.0005742,-0.4032,-0.2101,-0.0595,0.0606,0.1608]
b=[-0.01336,-0.02005,-0.1051,-0.1268,-0.1405,-0.1751]
c1=[-0.1132,-1.5012,35.6955,-268.7494,729.7605,-583.4203]
potB = ELJPotentialB{6}(a,b,c1)

ensemble = NVT(n_atoms)
move_strat = MoveStrategy(ensemble)


MoveStrategy{13, NVT}(NVT(13, 13, 0), ["atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove", "atommove"])

In [31]:
save_init(potB,ensemble,mc_params,temp)

In [23]:
# testfile = open("./checkpoint/testfile.data","w+")
# writeparams(testfile,mc_params,temp)
# writeensemble(testfile,ensemble)
# writepotential(testfile,potB)
# close(testfile)